In [10]:
import pandas as pd
#from coleta_dados import DadosFundamentalistas
from chat_bots import ChatLimpaResposta, ChatSentimento, ChatFundamentalistas
from coleta_dados import DadosFundamentalistas, DadosNoticiasBuscadorYahoo, DadosNoticiasGoogle
from tratando_dados import tratando_dados_fundamentalistas, TratarDadosNoticias
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
tic = 'BBDC3'
data_inicio = '2020-01-01'

dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)
trat = tratando_dados_fundamentalistas(tic, data_inicio)

In [3]:
dados = dados_fundamentalistas.dados_fundamentalistas_completo()

In [4]:
dados

,receita_liquida,resultado_bruto,ebit,amortizacao,ebitda,lucro_liquido,lucro_por_acao,datas,tic,capex_tres_meses,...,retorno_sobre_patrimonio_liquido_inicial,retorno_sobre_ativo_inicial,margem_bruta,margem_liquida,margem_ebit,margem_ebitda,giro_do_ativo_inicial,alavancagem_financeira,passivo_patrimonio_liquido,divida_liquida_ebitda
0,52500.0,15210.0,NaN,NaN,NaN,4870.00,NaN,2024-09-30,BBDC3,-10.25,...,0.088,0.008,0.277,0.061,NaN,NaN,0.13,12.07,11.07,NaN
1,51500.0,16940.0,NaN,NaN,NaN,4120.00,NaN,2024-06-30,BBDC3,-1190.00,...,0.078,0.007,0.277,0.054,NaN,NaN,0.13,12.17,11.17,NaN
2,52770.0,16890.0,NaN,NaN,NaN,4120.00,NaN,2024-03-31,BBDC3,-205.93,...,0.080,0.007,0.272,0.053,NaN,NaN,0.13,11.79,10.79,NaN
3,54110.0,17050.0,NaN,NaN,NaN,1440.00,NaN,2023-12-31,BBDC3,-393.47,...,0.090,0.008,0.271,0.057,NaN,NaN,0.14,11.59,10.59,NaN
4,56070.0,15920.0,NaN,NaN,NaN,3400.00,NaN,2023-09-30,BBDC3,-138.21,...,0.082,0.007,0.271,0.052,NaN,NaN,0.14,11.40,10.40,NaN
5,56850.0,17140.0,NaN,NaN,NaN,4020.00,NaN,2023-06-30,BBDC3,6.56,...,0.096,0.009,0.275,0.061,NaN,NaN,0.14,11.19,10.19,NaN
6,57430.0,17980.0,NaN,NaN,NaN,5400.00,NaN,2023-03-31,BBDC3,-77.89,...,0.119,0.011,0.294,0.076,NaN,NaN,0.14,11.25,10.25,NaN
7,53640.0,18270.0,NaN,NaN,NaN,785.85,NaN,2022-12-31,BBDC3,-178.48,...,0.140,0.012,0.320,0.090,NaN,NaN,0.14,11.37,10.37,NaN
8,53140.0,17100.0,NaN,NaN,NaN,5840.00,NaN,2022-09-30,BBDC3,-368.35,...,NaN,NaN,0.340,0.119,NaN,NaN,NaN,11.54,10.54,NaN
9,49280.0,18590.0,NaN,NaN,NaN,7140.00,NaN,2022-06-30,BBDC3,-172.53,...,NaN,NaN,0.379,0.134,NaN,NaN,NaN,11.10,10.10,NaN


In [5]:
trat

[Document(metadata={'receita_liquida': 52500.0, 'resultado_bruto': 15210.0, 'lucro_liquido': 4870.0, 'tic': 'BBDC3', 'capex_tres_meses': -10.25, 'fluxo_caixa_operacional': -6280.0, 'fluxo_caixa_investimentos': 8500.0, 'fluxo_caixa_financiamento': -4050.0, 'aumento_reducao_caixa_equivalentes': -1930.0, 'preço_lucro': '9.32', 'preço_receita_líquida': '0.57', 'preço_fco': '-3.30', 'preço_ativo_total': '0.07', 'market_cap_empresa': 144240.0, 'dividend_yield': 0.1076999999999999, 'ativo_total': 2020000.0, 'patrimonio_liquido': 167130.0, 'valor_patrimonial_acao': '15.70', 'retorno_sobre_patrimonio_liquido_inicial': 0.088, 'retorno_sobre_ativo_inicial': 0.008, 'margem_bruta': 0.277, 'margem_liquida': 0.061, 'giro_do_ativo_inicial': '0.13', 'alavancagem_financeira': '12.07', 'passivo_patrimonio_liquido': '11.07'}, page_content='2024-09-30'),
 Document(metadata={'receita_liquida': 51500.0, 'resultado_bruto': 16940.0, 'lucro_liquido': 4120.0, 'tic': 'BBDC3', 'capex_tres_meses': -1190.0, 'fluxo_c

In [6]:
query = 'Como esta a saude financeira da bbdc3'
response = ChatFundamentalistas(query=query, dados=trat,modelo_llm="deepseek-r1-distill-llama-70b")

In [7]:
response.split('\n\n')

['<think>\nOk, eu preciso analisar a saúde financeira da BBDC3 com base nos indicadores fornecidos. Primeiro, vou começar identificando as informações disponíveis. Vejo que há vários documentos com dados de diferentes datas, o que me permitirá analisar tendências ao longo do tempo.',
 'Vou começar olhando para a receita líquida. Observo que ela tem flutuado, mas parece que teve um pico em 2021 e está um pouco em declínio recentemente. Isso pode indicar que a empresa está enfrentando desafios na geração de receita.',
 'Em seguida, o lucro líquido. Vejo que em algumas datas o lucro foi positivo, mas em outras datas houve prejuízo. Isso é preocupante porque a rentabilidade irregular pode afetar a confiabilidade da empresa para os investidores.',
 'Margens brutas e líquidas: As margens parecem estáveis, o que é um ponto positivo. No entanto, a margem líquida está um pouco baixa, o que pode indicar pressão sobre os preços ou custos operacionais.',
 'Fluxos de caixa: Nota que o fluxo operaci

In [12]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

dados_yahoo = DadosNoticiasBuscadorYahoo(acao='BBDC3', options=chrome_options)


In [13]:
dados_ya = dados_yahoo.get_news(number_paginas=1)

Sem mais páginas!


In [14]:
ticker = 'BBDC3'
dados_google = TratarDadosNoticias(ticker, chrome_options)

In [16]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

ticker = 'BBDC3'
news_tratado = TratarDadosNoticias(ticker, chrome_options)
dados_new = news_tratado.clean_chat_html()

Sem mais páginas!
Poucas noticias encontrada para o ticker BBDC3 no Yahoo Finance. Buscando noticias no Google


Fetching pages: 100%|##########| 1/1 [00:03<00:00,  3.92s/it]


In [11]:
dados_new.split('New notice')

['\n ',
 '\n# Notícias sobre fiqe3\n\n### Artigo 1: [Título do Artigo]\n* fiqe3 lança novo serviço de [descrição do serviço]: "A fiqe3 anunciou a lançamento de um novo serviço que visa [benefícios do serviço]. Com isso, a empresa busca [objetivo da empresa]."\n* A empresa espera que o novo serviço [impacto positivo] para os clientes.\n\n### Artigo 2: [Título do Artigo]\n* A fiqe3 recebe [prêmio ou reconhecimento]: "A fiqe3 foi selecionada como uma das melhores empresas do ano de [ano], de acordo com [fonte]."\n* O prêmio foi dado devido ao [motivo do prêmio].\n\n### Artigo 3: [Título do Artigo]\n* A fiqe3 expande sua presença em [mercado ou região]: "A fiqe3 anunciou a abertura de uma nova filial em [localidade], aumentando sua presença em [mercado ou região]."\n* A empresa espera que a expansão [impacto positivo] para os clientes.\n\n### Artigo 4: [Título do Artigo]\n* A fiqe3 lança parceria com [empresa ou organização]: "A fiqe3 anunciou uma parceria com [empresa ou organização], vis

In [18]:
htt = news_tratado.get_news_google()

In [12]:
query = f'Qual é o sentimento sobre as notícias sobre a {ticker}'
dados_noticia = ChatSentimento(query=query, noticia=dados_new, modelo_llm="deepseek-r1-distill-llama-70b")

In [13]:
dados_noticia.split('\n\n')

['<think>\nOk, estou analisando as notícias sobre a empresa FIQUE3 para determinar o sentimento geral e decidir se é um bom momento para investir. Primeiro, vou passar por cada notícia e classificar o sentimento como positivo, neutro ou negativo, além de atribuir uma pontuação de -1 a +1.',
 'Na primeira notícia, a empresa lançou um novo serviço. Isso geralmente é positivo, pois indica crescimento e inovação. Vou dar uma pontuação de +0,8.',
 'A segunda notícia menciona que a FIQUE3 recebeu um prêmio. Prêmios são indicativos de reconhecimento e boa gestão, o que é positivo. Pontuação de +0,7.',
 'A terceira notícia fala sobre a expansão para um novo mercado. Expansão geográfica pode trazer mais receita e crescimento, então é positivo. Pontuação de +0,6.',
 'A quarta notícia trata de uma parceria. Parcerias estratégicas geralmente trazem benefícios e melhoram a posição no mercado. Pontuação de +0,5.',
 'Agora, as notícias sobre pagamento de dividendos e JCP. Pagamento de dividendos é po

In [1]:
import yfinance as yf
from finta import TA
from matplotlib import pyplot as plt
from coleta_dados import DadosIndicadoresTecnicos
from tratando_dados import TratandoDadosIndicadores

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
ticker = 'BBDC3'
preco_ticker = yf.Ticker(ticker.upper()+'.SA')

In [3]:
historico = preco_ticker.history(period='10Y', interval='1d')

In [4]:
trat_indi = TratandoDadosIndicadores(ticker=ticker)


In [ ]:
dados_indicadores =   trat_indi.indicadores_data_loader()


In [20]:
dados_indicadores

[Document(metadata={'20 period SMA': 12.861459541320801, '100 period SMA': 12.408729629516602, '200 period SMA': 12.493582792282105, '14 period RSI': 65.11512964973593, 'MACD': 0.5564387332552538, 'SIGNAL': 0.39432785885921867, 'BB_UPPER': 14.808979902430421, 'BB_MIDDLE': 12.861459541320801, 'BB_LOWER': 10.913939180211182, 'pivot': 14.08501134346632, 's1': 13.871697997012838, 's2': 13.758767316657515, 's3': 13.545453970204033, 's4': 13.33214062375055, 'r1': 14.197942023821643, 'r2': 14.411255370275125, 'r3': 14.524186050630448, 'r4': 14.63711673098577, 'VWAP.': 12.276989216282267, '14 period ADX.': 31.222277920705444, 0: 12.431798115603504}, page_content='2020-11-19'),
 Document(metadata={'20 period SMA': 12.91506814956665, '100 period SMA': 12.429151248931884, '200 period SMA': 12.473884525299072, '14 period RSI': 63.3877309953973, 'MACD': 0.5496813756640506, 'SIGNAL': 0.4253985622201851, 'BB_UPPER': 14.915181750108696, 'BB_MIDDLE': 12.91506814956665, 'BB_LOWER': 10.914954549024605, '

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from chat_bots import get_secret_key
from pydantic import SecretStr
from typing import List

try:
    api_secret_groq = get_secret_key("GROQ_API_KEY")
except KeyError as exc:
    raise ValueError("API key inválida ou não definida") from exc


def ChatAnaliseTecnica(
    query: str,
    dados: List[str],
    api_secret: SecretStr | None = api_secret_groq,
    temperature: float = 0.5,
    modelo_llm: str = "llama-3.3-70b-versatile",
) -> str:
    prompt = PromptTemplate(
        input_variables=["query", "dados"],
        template="""
        
        Você é um analista técnico especializado na interpretação de indicadores financeiros para identificar tendências, pontos de entrada e saída no mercado. Utilize os seguintes dados técnicos para fornecer uma análise detalhada da situação atual do ativo e uma possível estratégia de trading:  

        ### Indicadores Disponíveis:  
        - Médias Móveis  
          - 20 períodos: 
          - 100 períodos: 
          - 200 períodos: 
        - Índice de Força Relativa (RSI - 14 períodos)
        - MACD e Linha de Sinal:  
          - MACD:  
          - Sinal: 
        - Bandas de Bollinger:  
          - Superior: 
          - Média: 
          - Inferior:  
        - Pontos de Suporte e Resistência (Pivot Points):  
          - Ponto Pivot:  
          - Suportes: 
          - Resistências: 
        - VWAP (Preço Médio Ponderado pelo Volume)  
        - ADX (Índice de Direção Média - 14 períodos) 

        ### Objetivo da Análise:  
        1. **Identificação da Tendência:**  
          - O preço está acima ou abaixo das médias móveis?  
          - As médias móveis indicam uma tendência de alta ou baixa?  
          - O ADX indica uma tendência forte (>25) ou fraca (<25)?  

        2. **Força do Mercado:**  
          - O RSI sugere sobrecompra (>70) ou sobrevenda (<30)?  
          - O MACD está cruzando a linha de sinal, sugerindo um possível ponto de entrada/saída?  

        3. **Suporte e Resistência:**  
          - O preço atual está próximo de um suporte ou resistência relevante?  
          - As Bandas de Bollinger indicam alta volatilidade (expansão) ou baixa volatilidade (contração)?  

        4. **Possível Estratégia:**  
          - Se tendência de alta: Identificar pontos de compra e alvos de lucro.  
          - Se tendência de baixa: Identificar pontos de venda e alvos de queda.  
          - Se mercado lateralizado: Sugerir estratégias de reversão ou rompimento.  

        Baseando-se nesses fatores, forneça uma análise detalhada da situação atual do ativo e uma recomendação de ação (comprar, vender ou aguardar).  

              
                        
                                                                                                   
        REQUESTED ANALYSIS:
        Question: {query}
        Financial Data: {dados}
        
        
        Always response the questions in Portuguese.

      """,
    )

    model = ChatGroq(
        api_key=api_secret,
        model=modelo_llm,
        # model="llama-3.2-11b-vision-preview",
        temperature=temperature,
        stop_sequences=None,
    )

    llm_chain = prompt | model | StrOutputParser()

    response = llm_chain.invoke(input={"query": query, "dados": dados})

    return response

In [ ]:
ChatAnaliseTecnica(
    query=f"Qual é a tendencia sobre a empresa {ticker}",
    dados=dados_indicadores,
    modelo_llm="deepseek-r1-distill-llama-70b"
)